In [4]:
import os
from docx import Document
import pandas as pd
import re

import pymorphy2
morph = pymorphy2.MorphAnalyzer(lang='ru')
import string
import nltk
from nltk.corpus import stopwords 
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.stem.porter import PorterStemmer
from nltk import sent_tokenize, word_tokenize
from nltk.corpus import stopwords
import string

from pymystem3 import Mystem

In [5]:
def all_to_first_upper_letter(word):
    set_ul_word = list(set([l.isupper() for l in word.replace('.', '')]))
    if len(set_ul_word) == 1 and set_ul_word[0] == True:
        return word[0]+word[1:].lower()
    else:
        return word
    
all_to_first_upper_letter('ВВЕДЕНИЕ')

'Введение'

In [6]:
def all_to_first_upper_letter_of_words(sent):
    sent = sent.replace('\n', ' ').replace('\t', ' ')
    return ' '.join([all_to_first_upper_letter(word) for word in sent.split() if word != ''])

sent = 'ФЕДЕРАЛЬНОЕ ГОСУДАРСТВЕННОЕ АВТОНОМНОЕ ОБРАЗОВАТЕЛЬНОЕ а УЧРЕЖДЕНИЕ ВЫСШЕГО ОБРАЗОВАНИЯ\n“Национальный исследовательский университет ИТМО”'
all_to_first_upper_letter_of_words(sent)

'Федеральное Государственное Автономное Образовательное а Учреждение Высшего Образования “Национальный исследовательский университет ИТМО”'

In [7]:
def simple_sentence_preproc(sentence):
    sentence = sentence.replace('\t', ' ').replace('\n', ' ').replace(' ..', '.')
    sentence = all_to_first_upper_letter_of_words(sentence)
    return ''.join(re.findall('[А-Яа-я\. ]', sentence)).replace('  ', ' ').replace(' .', '.').strip()

sent = 'ФЕДЕРАЛЬНОЕ ГОСУДАРСТВЕННОЕ АВТОНОМНОЕ ОБРАЗОВАТЕЛЬНОЕ УЧРЕЖДЕНИЕ в  ВЫСШЕГО ОБРАЗОВАНИЯ\n“Национальный исследовательский университет ИТМО”.'
simple_sentence_preproc(sent)    

'Федеральное Государственное Автономное Образовательное Учреждение в Высшего Образования Национальный исследовательский университет ИТМО.'

In [20]:
def get_docx_sections_texts(document):
    dict_sections_texts = {}
    paragraphs = document.paragraphs
    section_name = paragraphs[0].text
    section_name = simple_sentence_preproc(section_name)  
    dict_sections_texts[section_name] = []
    n = 1
    for line, p in enumerate(paragraphs[1:]):
        section_alignment = str(p.paragraph_format.alignment).split()[0] 
        section_text = p.text
        section_text = simple_sentence_preproc(section_text)  
        
        #print(n, section_alignment, section_text)
        if section_alignment == 'CENTER':
            section_name = section_text
            dict_sections_texts[section_name] = []

        elif section_alignment != 'CENTER':
            if section_text != '':
                dict_sections_texts[section_name].append(section_text)
            
        n = n+1
    
    dict_sections_texts_ = {}
    for key, lst_sents in dict_sections_texts.items():
        key = key.lower()
        if key == ''  or 'картинка' in key or 'рисунок' in key or 'таблица' in key:
            continue  
        dict_sections_texts_[key] = ' '.join(lst_sents).replace(' ..', '.').replace(' .', '.')
                         
    return dict_sections_texts_

In [9]:
path_docs = "C:/Users/JM 505 Computers/Desktop/Repetitor/Ульяна/Диплом/ВКР_2020/"
lst_paths_files = []
for root, dirs, files in os.walk(path_docs):
    for name in files:
        path_file = os.path.join(root, name)
        lst_paths_files.append(path_file)

In [21]:
%%time
lst_dict_sections_texts = []
lst_papers = []
lst_paths_docs = []
for path_file in lst_paths_files[1:]:
    path_file = path_file.replace('\\', '/')
    f = open(path_file, 'rb')
    document = Document(f)
    paper = [simple_sentence_preproc(p.text) for p in document.paragraphs]
    paper = [sent for sent in paper if sent != '']
    lst_papers.append(paper)
    lst_dict_sections_texts.append(get_docx_sections_texts(document))
    lst_paths_docs.append(path_file)

Wall time: 55.8 s


In [11]:
df_docs = pd.DataFrame()
df_docs['path_doc'] = lst_paths_docs
df_docs['title'] = df_docs['path_doc'].apply(lambda x: x.split('/')[-1])
df_docs['text_paper'] = lst_papers

df_docs['dict_sections_texts'] = lst_dict_sections_texts

df_docs['faculty_department'] = df_docs['path_doc'].apply(lambda x: x.replace(path_docs, '').split('/')[:-1])
df_docs['faculty'] = df_docs['faculty_department'].apply(lambda x: x[0])
df_docs['department'] = df_docs['faculty_department'].apply(lambda x: x[1] if len(x)>1 else x[0])

df_docs['degree'] = ['bachelor' for _ in range(len(df_docs))]
df_docs = df_docs.drop(columns = 'path_doc')
print(len(df_docs))
df_docs.head()

359


,title,text_paper,dict_sections_texts,faculty_department,faculty,department,degree
0,Анализ психологических типов сотрудников для м...,[Министерство науки и высшего образования Росс...,{'министерство науки и высшего образования рос...,"[ФБИТ, ПБКС]",ФБИТ,ПБКС,bachelor
1,Исследование влияния алгоритма стохастического...,[Министерство науки и высшего образования Росс...,{'министерство науки и высшего образования рос...,"[ФБИТ, ПБКС]",ФБИТ,ПБКС,bachelor
2,Разработка VPN-приложения для мобильных устрой...,[Министерство науки и высшего образования Росс...,{'министерство науки и высшего образования рос...,"[ФБИТ, ПБКС]",ФБИТ,ПБКС,bachelor
3,Разработка автоматизированной системы защиты и...,[Министерство науки и высшего образования Росс...,{'министерство науки и высшего образования рос...,"[ФБИТ, ПБКС]",ФБИТ,ПБКС,bachelor
4,Разработка автоматизированной системы монитори...,[Министерство науки и высшего образования Росс...,{'министерство науки и высшего образования рос...,"[ФБИТ, ПБКС]",ФБИТ,ПБКС,bachelor


In [24]:
df_docs['dict_sections_texts'].values[3]

{'министерство науки и высшего образования российской федерации федеральное государственное автономное образовательное учреждение высшего образования национальный исследовательский университет итмо': '',
 'выпускная квалификационная работа': '',
 'разработка автоматизированной системы защиты информаци в системе медикосоциальной экспертизы': 'Автор Пенин Андрей Семенович Фамилия Имя Отчество Подпись Направление подготовки специальность.. код наименование',
 'информационная безопасности': 'Квалификация бакалавр бакалавр магистр инженер Руководитель Вкр Исаев А.С. к.т.н. Фамилия И. О. ученое звание степень Подпись',
 'санктпетербург г.': 'Обучающийся Пенин Андрей Семенович',
 'фио полностью': 'Группа Факультетинституткластер Бит Направленность профиль специализация Организация и технология защиты информации Вкр принята г. Оригинальность ВКР Вкр выполнена с оценкой Дата защиты  июня  г. Секретарь Гэк Коваль Елена Николаевна ФИО подпись Листов хранения',
 'демонстрационных материаловчертеже

In [13]:
%%time

russian_stopwords = stopwords.words("russian")
russian_stopwords = set(russian_stopwords + ['который', 'таблица', 'рисунок', 
                                             'тот', 'также', 'этот', 'это', 
                                             'такой', 'каждый', 'другой'])


def sentence_preproc(sentence):

    sentence = ''.join([ch for ch in sentence if ch not in string.punctuation])
    sentence = re.sub(r'[^а-яА-Я]', ' ', sentence).strip().replace('  ', '')
    sentence = sentence.split()

    sentence_new = [morph.parse(word)[0].normal_form for word in sentence if word not in russian_stopwords and \
                str(morph.parse(word)[0].tag) != 'UNKN']
    sentence_new = ' '.join(sentence_new)
    return sentence_new



def get_main_preproc_text(dict_sections_texts):
    text = ''
    for sents in dict_sections_texts.values():
        if sents != '':
            text = text + " " + sents 
      
    return sentence_preproc(text)


df_docs['main_preproc_text'] = df_docs['dict_sections_texts'].map(get_main_preproc_text)

Wall time: 8min 28s


In [14]:
df_docs['main_preproc_text'].values[0]

'федеральный государственный автономный образовательный учреждение высокий национальный исследовательский университет итмый утверждать руководитель оп канжелевый фамилия подпись декабрячь автор иванов александр сергеевич направление подготовка специальность код наименование информационный безопасность квалификация бакалавр руководитель канжелевый фамилия и о учёный звание степень подпись руководитель оп канжелевый маяга обучаться иванов асгруппа факультет битый фамилия и о направленность профиль специализация комплексный защита объект информатизация консультант ы фамилия и о учёный звание степень подпись фамилия и о учёный звание степень подпись принятамаяга оригинальность выполнить оценка дата защитыиюняга секретарь кузнецов ая лист хранение демонстрационный материаловчертёж хранение обучаться иванов александра сергеевич группа факультет битый руководитель канжелевый юрий алексеевич ктехнна старший научный сотрудник доцент квалификационный категория доцент практика факультет безопасно

In [23]:
df_docs['dict_sections_texts'].values[0].keys()

dict_keys(['министерство науки и высшего образования российской федерации', 'образования', 'национальный исследовательский университет итмо', 'выпускная квалификационная работа', 'анализ психологических типов сотрудников для минимизации утечки конфиденциальной информации средствами социальной инженерии', 'фамилия имя отчество подпись', 'бакалавр магистр', 'к защите допустить', 'фамилия и.о. ученое звание степень подпись', 'санктпетербург г.', 'фио подпись', 'министерство науки и высшего образования российской федерации федеральное государственное автономное образовательное учреждение высшего образования', 'задание', 'на выпускную квалификационную работу', 'аннотация', 'выпускной квалификационной работы', 'характеристика выпускной квалификационной работы', 'библиографическое описание публикаций', 'содержание', 'введение', 'выводы', 'заключение', 'актуальность выбранной проблематики связана с тем что взаимодействие специалиста по обеспечению защиты данных с сотрудниками имеющими к ним до

In [25]:
df_docs.to_csv('VKR_docs.csv', index = False)
df_docs.to_excel('VKR_docs.xlsx', index = False)